We use this notebook to investigate the pixel shift caused by ee.export()

In [1]:
import os
service_prefix = ''
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{service_prefix}/proxy/{{port}}"
import sys
import ee
import geemap
import numpy as np
import tqdm
import cv2
import seaborn as sns
from datetime import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from omegaconf import DictConfig, OmegaConf
import hydra
import cv2
from eval.surfrad import get_emis_at, read_surfrad_file_from_url, get_surfrad_surf_temp_at
from util.ee_utils import get_landsat_capture_time, get_landsat_lst, cvt_lat_lon_to_path_row, query_geotiff

## Original half a pixel shift

In [2]:
vis = {
  'min': 280,
  'max': 340,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}
config = OmegaConf.load('../config/surfrad.yaml')
station_id = 'BND'
date_ = '20170611'
lon = config['stations'][station_id]['Longitude']
lat = config['stations'][station_id]['Latitude']
scene_id = config['stations'][station_id]['scene_id']
Map = geemap.Map()
Map.add_basemap('HYBRID')

img_path = f'/home/yuhaoliu/Code/UrbanSurfTemp/data/{station_id}/lst/LC08_ST_B10_{date_}.tif'
# output_path = '/home/yuhaoliu/Code/UrbanSurfTemp/data/Austin/output_referenced_f75/lst/20170303.tif'
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_{scene_id}_{date_}')
Map.add_raster(img_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Debugging the issue by exporting to Google Drive 

```python
geemap.ee_export_image_to_drive(img, description="entire_image_all_bands", folder="export", region=None, scale=30)
geemap.ee_export_image_to_drive(img.select('ST_B10'), description="entire_image_bt10", folder="export", region=None, scale=30)
geemap.ee_export_image_to_drive(img.select('ST_B10').multiply(0.00341802).add(149), description="entire_image_scaled", folder="export", region=None, scale=30)
geemap.ee_export_image_
```

In [2]:
vis = {
  'min': 280,
  'max': 340,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}
config = OmegaConf.load('../config/surfrad.yaml')
station_id = 'BND'
date_ = '20170611'
lon = config['stations'][station_id]['Longitude']
lat = config['stations'][station_id]['Latitude']
scene_id = config['stations'][station_id]['scene_id']
Map = geemap.Map()
Map.add_basemap('HYBRID')

image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_{scene_id}_{date_}')
downloaded_path = f'/home/yuhaoliu/Code/UrbanSurfTemp/data/{station_id}/lst/LC08_ST_B10_{date_}.tif'
# entire_bt10 = '/home/yuhaoliu/Data/ISLAND/gdrive/entire_image_bt10.tif'
entire_scaled = '/home/yuhaoliu/Data/ISLAND/gdrive/entire_image_scaled.tif'
cropped_scaled = '/home/yuhaoliu/Data/ISLAND/gdrive/cropped_scaled.tif'
Map.add_raster(downloaded_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.add_raster(cropped_scaled, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='cropped_scaled')
Map.add_raster(entire_scaled, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='entire_scaled')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
lon, lat = -88.261, 40.145

landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')

direct_download_lst = query_geotiff(lon, lat, downloaded_path)
print(f'Direct download GeoTIFF LST = {direct_download_lst}')

gdrive_lst = query_geotiff(lon, lat, cropped_scaled)
print(f'Gdrive GeoTIFF LST = {gdrive_lst}')

LANDSAT LST = 334.53696163999996 K
Direct download GeoTIFF LST = 331.79912762000004
Gdrive GeoTIFF LST = 334.53696163999996


When ee.Image are exported to Google Drive, even when there is cropping applied, there is no pixel shift.

## Loading modified images

In [2]:
vis = {
  'min': 280,
  'max': 340,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}
config = OmegaConf.load('../config/surfrad.yaml')
station_id = 'BND'
date_ = '20170611'
lon = config['stations'][station_id]['Longitude']
lat = config['stations'][station_id]['Latitude']
scene_id = config['stations'][station_id]['scene_id']
Map = geemap.Map()
Map.add_basemap('HYBRID')

img_path = f'/home/yuhaoliu/Code/UrbanSurfTemp/data/{station_id}/lst/LC08_ST_B10_{date_}.tif'
aligned_path = '/home/yuhaoliu/Code/UrbanSurfTemp/tmp/align.tif'
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_{scene_id}_{date_}')
Map.add_raster(img_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.add_raster(aligned_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='aligned_lst')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
lon, lat = -88.261, 40.145

landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')

direct_download_lst = query_geotiff(lon, lat, img_path)
print(f'Direct download GeoTIFF LST = {direct_download_lst}')

aligned_lst = query_geotiff(lon, lat, aligned_path)
print(f'Aligned GeoTIFF LST = {aligned_lst}')

LANDSAT LST = 334.53696163999996 K
Direct download GeoTIFF LST = 331.79912762000004
Aligned GeoTIFF LST = 334.53696163999996


Success!

In [2]:
vis = {
  'min': 270,
  'max': 320,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}
config = OmegaConf.load('../config/surfrad.yaml')
station_id = 'PSU'
date_ = '20170322'
lon = config['stations'][station_id]['Longitude']
lat = config['stations'][station_id]['Latitude']
scene_id = config['stations'][station_id]['scene_id']
Map = geemap.Map()
Map.add_basemap('HYBRID')

img_path = f'/home/yuhaoliu/Code/UrbanSurfTemp/data/{station_id}/lst/LC08_ST_B10_{date_}.tif'
aligned_path = '/home/yuhaoliu/Code/UrbanSurfTemp/tmp/align_2.tif'
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_{scene_id}_{date_}')
Map.add_raster(img_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.add_raster(aligned_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='aligned_lst')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…